In [ ]:
import os
import datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import concatenate

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # don't pre-allocate memory; allocate as-needed
config.gpu_options.per_process_gpu_memory_fraction = 0.5 # limit memory to be allocated
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
'''
    Given a dataset, first building the Trie to store the frequency count information.
'''
import pickle
import pygtrie

trie = pygtrie.StringTrie(separator=',')

# Function below increments the count of key in dictionary.
def increment_count(song):
    if(trie.has_key(song)):
        trie[song]+=1
    else:
        trie[song]=1

train_file = open('ml-1m/train.csv', 'r')

# Looping through our training dataset
for line in train_file:
    songs_array = line.strip().split(',')
    num_songs = len(songs_array)
    
    #print(num_songs)
    
    for i in range(num_songs):
        song = songs_array[i]
        
        increment_count(song)
        
    for i in range(num_songs-1):
        song_1 = songs_array[i] 
        song_2 = songs_array[i + 1] 
        song_set = song_1 + ',' + song_2
        
        increment_count(song_set)
            
    for i in range(num_songs-2):
        song_1 = songs_array[i] 
        song_2 = songs_array[i + 1] 
        song_3 = songs_array[i + 2] 
        song_set = song_1 + ',' + song_2 + ',' + song_3
        
        increment_count(song_set)
        
    for i in range(num_songs-3):
        song_1 = songs_array[i] 
        song_2 = songs_array[i + 1] 
        song_3 = songs_array[i + 2] 
        song_4 = songs_array[i + 3] 
        song_set = song_1 + ',' + song_2 + ',' + song_3 + ',' + song_4
        
        increment_count(song_set)

train_file.close()

# Saving Trie to disk.
pickle.dump(trie, open('ml-1m/ml1m.trie', 'wb'))

In [ ]:
#Loading trie from disk.
import pickle

trie = pickle.load(open( "ml-1m/ml1m.trie", "rb" ))

In [ ]:
all_item_pairs = list(trie.items())

total_1_gram_count = 0
for pair in all_item_pairs:
    item_list = pair[0].split(',')
    if(len(item_list) == 1):
        total_1_gram_count+=pair[1]

'''
    Given song_prefix (a ordered sequence of items) & candidate_song, get_alpha_beta_values
    returns the corresponding alpha & beta values as follows:
    alpha: (song_prefix, candidate_song)
    beta: (song_prefix, ~candidate_song)
'''
def get_alpha_beta_values(song_prefix, candidate_song):
    
    if(song_prefix == ''):
        alpha = 0
        
        list_tuples = list(trie.iteritems(prefix=candidate_song))
        for pair in list_tuples:
            song_list = pair[0].split(',')
            #print(song_list)
            if(len(song_list) == 1):
                if(song_list[0] == candidate_song):
                    alpha = pair[1]    
        return (alpha, total_1_gram_count - alpha)
    
    num_prefix = len(song_prefix.split(','))
    
    if(trie.has_subtrie(song_prefix) == False):
        return (0, 0) # For now
    
    list_tuples = list(trie.iteritems(prefix=song_prefix))
    
    # list_tuples will contain the (key, count) pairs
    alpha = 0
    beta = 0
    for pair in list_tuples:
        song_list = pair[0].split(',')
        
        # song_prefix can be same as the song_list        

        if(len(song_list) == (num_prefix + 1)):
            if(song_list[num_prefix] == candidate_song):
                alpha+=pair[1]
            else:
                beta+=pair[1]
    
    return (alpha, beta)

In [ ]:
'''
    Returns the most popular item after prefix 'three_songs'
    (Implements the max approach).
'''
def get_most_popular(three_songs):
    list_tuples = list(trie.iteritems(prefix=three_songs))
    freq_song_list = []
    for pair in list_tuples:
        song_list = pair[0].split(',')
        #print(song_list)
        if(len(song_list) == 4):
            freq_song_list.append((song_list[3], pair[1]))
    
    freq_song_list.sort(key=lambda x: x[1], reverse=True)
    
    return freq_song_list

In [ ]:
'''
    Generating the training file for the Neural Networks.
    Generation Process Pseudocode:
        Use window approach to attain a context of 4 items observed in a user interaction history.
        For each sequence, we generate a positive & negative instance where the observed sequence 
            is the positive sequence & negative sequence is attained as mentioned in the paper.
        Each line of the training file for neural networks consists of [8, 8] statistics.
'''
input_file = open('ml-1m/train.csv', 'r')

output_file = open('ml-1m/nn_train_v4.csv', 'w')

count = 0
special_count = 0

for line in input_file:
    
    if(count%1000 == 0):
        print(count)
    
    count+=1
    
    items_array = line.strip().split(',')
    array_length = len(items_array)
    
    if(array_length < 4):
        continue
        
    # Going over the song_array with window of size 4
    for i in range(0, array_length-3, 4):
        # [i -> i+3] both inclusive
        a = items_array[i]
        b = items_array[i+1]
        c = items_array[i+2]
        d = items_array[i+3]
        
        # Getting the following eight counts:
            #d        #(~d)
            #cd       #(c~d)
            #bcd      #(bc~d)
            #abcd     #(abc~d)
        (d_alpha, d_beta) = get_alpha_beta_values('', d)
        (cd_alpha, cd_beta) = get_alpha_beta_values(c, d)
        (bcd_alpha, bcd_beta) = get_alpha_beta_values(b + ',' + c, d)
        (abcd_alpha, abcd_beta) = get_alpha_beta_values(a + ',' + b + ',' + c, d)
        
        # Save the above numbers to a file.
        pos_str_to_write = ','.join(map(str, [d_alpha, cd_alpha, bcd_alpha, abcd_alpha, \
                  d_beta, cd_beta, bcd_beta, abcd_beta]))
        
        
        # Negative Example
        e = ''
        sorted_freq_list = get_most_popular(a + ',' + b + ',' + c)
        
        if(len(sorted_freq_list) < 2):
            # Only 1 element in the list.
            # Roll back and find the negative example with b,c,d.
            sorted_freq_list = get_most_popular(b + ',' + c)
        
        if(len(sorted_freq_list) < 2):
            sorted_freq_list = get_most_popular(c)
        
        if(len(sorted_freq_list) < 2):
            special_count+=1
            continue
    
        if(sorted_freq_list[0][0] == d):
            # Choose the next most popular song
            e = sorted_freq_list[1][0]
        else:
            # Choose the most popular song
            e = sorted_freq_list[0][0]

        
        # Found our negative example: a b c e.
        (e_alpha, e_beta) = get_alpha_beta_values('', e)
        (ce_alpha, ce_beta) = get_alpha_beta_values(c, e)
        (bce_alpha, bce_beta) = get_alpha_beta_values(b + ',' + c, e)
        (abce_alpha, abce_beta) = get_alpha_beta_values(a + ',' + b + ',' + c, e)
        
        # Save the above numbers to a file.
        neg_str_to_write = ','.join(map(str, [e_alpha, ce_alpha, bce_alpha, abce_alpha, \
                  e_beta, ce_beta, bce_beta, abce_beta]))

        #print('============')
        output_file.write( pos_str_to_write + ',' + neg_str_to_write + '\n')

input_file.close()
output_file.close()

In [ ]:
# Train the neural networks.

from numpy import genfromtxt

positive_input = keras.Input(shape=(8,), name="positive_inputs")
negative_input = keras.Input(shape=(8,), name="negative_inputs")

f_alpha = layers.Dense(1000, input_shape=(8,))
f_beta = layers.Dense(1000, input_shape=(8,))

temp_alpha = layers.Dense(500, input_shape=(1000,))
temp_beta = layers.Dense(500, input_shape=(1000,))

alpha_inter_layer = layers.Dense(1, name = "alpha_intermediate")
beta_inter_layer = layers.Dense(1, name = "beta_intermediate")

'''
    Architecture for the alpha & beta functions:

    8 -> 1000 -> 500 -> 1 # alpha
    8 -> 1000 -> 500 -> 1 # beta
'''

f_alpha_pos = alpha_inter_layer(temp_alpha(f_alpha(positive_input)))
f_beta_pos = beta_inter_layer(temp_beta(f_beta(positive_input)))

f_alpha_neg = alpha_inter_layer(temp_alpha(f_alpha(negative_input)))
f_beta_neg = beta_inter_layer(temp_beta(f_beta(negative_input)))

pos_num = (f_alpha_pos+1.0)
pos_denom = (f_alpha_pos+f_beta_pos+2.0)

neg_num = (f_alpha_neg+1.0)
neg_denom = (f_alpha_neg+f_beta_neg+2.0)

positive_mean = pos_num/pos_denom
negative_mean = neg_num/neg_denom

pos_negative_difference = keras.layers.Subtract(name="final_output")\
            ([positive_mean, negative_mean])

global_model = keras.Model(
    inputs=[positive_input, negative_input],
    outputs=[pos_negative_difference]
)

global_model.compile(
    optimizer = keras.optimizers.Adagrad(),
    loss = tf.keras.losses.Hinge()
)

'''
    Hinge Loss:
        max(1-y_true*y_pred, 0)
'''

train_data = genfromtxt('ml-1m/nn_train.csv', delimiter=',')
(num_bcd_instances, _) = train_data.shape

train_data = np.log(train_data + 0.1)

(pos_data, neg_data) = np.split(train_data, 2, axis = 1)

true_output = 1000*np.ones((num_bcd_instances, 1))

global_model.fit(
    {"positive_inputs": pos_data, "negative_inputs": neg_data},
    {"final_output": true_output},
    epochs = 20,
    batch_size = 50
)

In [ ]:
# Further training can be done below.
global_model.fit(
    {"positive_inputs": pos_data, "negative_inputs": neg_data},
    {"final_output": true_output},
    epochs = 20,
    batch_size = 200
)

In [ ]:
global_alpha = keras.Model(inputs=global_model.inputs, name="neutral_input",\
                             outputs=global_model.get_layer('alpha_intermediate').output)
global_beta = keras.Model(inputs=global_model.inputs, name="neutral_input",\
                             outputs=global_model.get_layer('beta_intermediate').output)

In [ ]:
global_model.save('ml-1m/global_ml1m_log_v6.h5', save_format='h5')

In [ ]:
# Implements the most popular given context baseline.
def most_freq_4gram(song_1, song_2, song_3):
    if(trie.has_subtrie(song_1+','+song_2+','+song_3)):
        #Get the one with the max value and return it
        list_tuples = list(trie.iteritems(prefix=song_1+','+song_2+','+song_3))
        
        # Filter out the length 3 tuples
        filtered_list_tuples = []
        for pair in list_tuples:
            if(len(pair[0].split(','))>3):
                filtered_list_tuples.append(pair)
                
        (max_key, max_count) = max(filtered_list_tuples, key=lambda item:item[1])
        max_key_list = max_key.split(',')
        
        temp_list = sorted(filtered_list_tuples, key=lambda tup: tup[1], reverse=True)[:20]
        return [pair[0].split(',')[3] for pair in temp_list]

    
    if(trie.has_subtrie(song_2+','+song_3)):
        #Get the one with the max value and return it
        list_tuples = list(trie.iteritems(prefix=song_2+','+song_3))
        
        # Filter out the length 3 tuples
        filtered_list_tuples = []
        for pair in list_tuples:
            if(len(pair[0].split(','))>2):
                filtered_list_tuples.append(pair)
                
        (max_key, max_count) = max(filtered_list_tuples, key=lambda item:item[1])
        max_key_list = max_key.split(',')
            
        temp_list = sorted(filtered_list_tuples, key=lambda tup: tup[1], reverse=True)[:20]
        return [pair[0].split(',')[2] for pair in temp_list]

    if(trie.has_subtrie(song_3)):
        #Get the one with the max value and return it
        list_tuples = list(trie.iteritems(prefix=song_3))
        
        # Filter out the length 3 tuples
        filtered_list_tuples = []
        for pair in list_tuples:
            if(len(pair[0].split(','))>1):
                filtered_list_tuples.append(pair)
                
        (max_key, max_count) = max(filtered_list_tuples, key=lambda item:item[1])
        max_key_list = max_key.split(',')
        
        temp_list = sorted(filtered_list_tuples, key=lambda tup: tup[1], reverse=True)[:20]
        return [pair[0].split(',')[1] for pair in temp_list]

    return sorted_most_popular_song_tuple_list[:20]

In [ ]:
def get_most_popular_prefix(songs_prefix):
    list_tuples = list(trie.iteritems(prefix=songs_prefix))
    prefix_length = len(songs_prefix.split(','))
    
    interested_pair_list = []
    for pair in list_tuples:
        song_list = pair[0].split(',')
        if(len(song_list) == (prefix_length+1) ):
            #Add to the list
            interested_pair_list.append((song_list[prefix_length], pair[1]))
    
    # Now return the most frequent one.
    if(len(interested_pair_list) == 0):
        return ''
    
    interested_pair_list.sort(key=lambda x: x[1], reverse=True)
    return interested_pair_list[0][0]

total_1_gram_count = 0

most_popular_song_tuple_list = []

for pair in all_item_pairs:
    song_list = pair[0].split(',')
    if(len(song_list) == 1):           
        most_popular_song_tuple_list.append((song_list[0], pair[1]))
        total_1_gram_count+=pair[1]

sorted_most_popular_song_tuple_list = \
    sorted(most_popular_song_tuple_list, key=lambda tup: tup[1], reverse=True)
        

'''
    Function below returns the most popular 4th songs given the first 3 songs.
'''
def get_most_popular(three_songs):
    list_tuples = list(trie.iteritems(prefix=three_songs))
    freq_song_list = []
    for pair in list_tuples:
        song_list = pair[0].split(',')
        #print(song_list)
        if(len(song_list) == 4):
            freq_song_list.append((song_list[3], pair[1]))
    
    freq_song_list.sort(key=lambda x: x[1], reverse=True)
    
    return freq_song_list

def get_top_k(list_pairs, k):
    # Given a list of pairs returns the top k values
    if(k > len(list_pairs)):
        k = len(list_pairs)

    list_pairs.sort(key=lambda x: x[1], reverse=True)
    return [pair[0] for pair in list_pairs[:k]]

'''
    Candidate generation is recall focused.
    Given a prefix of items: 'song_1, song_2, song_3', candidates are recommended based on their
    appearance with this prefix in the training data.
    Higher priority is given to candidates from a longer prefix.
'''
def generate_candidates(song_1, song_2, song_3):
    
    candidates = []
    threshold_total_candidates = 10
    
    subset_candidates = []
    if(trie.has_subtrie(song_1+','+song_2+','+song_3)):
        #Get the one with the max value and return it
        list_tuples = list(trie.iteritems(prefix=song_1+','+song_2+','+song_3))
        for pair in list_tuples:
            if(len(pair[0].split(','))>3):
                subset_candidates.append( (pair[0].split(',')[3], pair[1]) )
    
    if(len(subset_candidates) < threshold_total_candidates):
        candidates.extend([item[0] for item in subset_candidates])
    else:
        return get_top_k(subset_candidates, threshold_total_candidates)
        
    subset_candidates = []
    if(trie.has_subtrie(song_2+','+song_3)):
        # Get the one with the max value and return it
        list_tuples = list(trie.iteritems(prefix=song_2+','+song_3))
        for pair in list_tuples:
            if(len(pair[0].split(','))>2):
                subset_candidates.append( (pair[0].split(',')[2], pair[1]) )
    
    if( (len(subset_candidates) + len(candidates))  < threshold_total_candidates):
        candidates.extend([item[0] for item in subset_candidates])
    else:
        candidates.extend(get_top_k(subset_candidates, threshold_total_candidates - len(candidates)))
        return candidates
        
    subset_candidates = []
    if(trie.has_subtrie(song_3)):
        #Get the one with the max value and return it
        list_tuples = list(trie.iteritems(prefix=song_3))
        for pair in list_tuples:
            if(len(pair[0].split(','))>1):
                subset_candidates.append( (pair[0].split(',')[1], pair[1]) )
    
    if( (len(subset_candidates) + len(candidates))  < threshold_total_candidates):
        candidates.extend([item[0] for item in subset_candidates])
        return candidates
    
    candidates.extend(get_top_k(subset_candidates, threshold_total_candidates - len(candidates)))
    return candidates

In [ ]:
# Implements our Bayesian approach.
def predict_correct_song(song_1, song_2, song_3, max_song):
    
    candidates = []

    candidates = generate_candidates(song_1, song_2, song_3)
    
    # Result from max frequency.
    if(max_song != ""):
        candidates.append(max_song)
            
    #Stores the score for each candidate
    to_return_map = {}
    
    for candidate_song in candidates:
        
        (x_alpha, x_beta) = get_alpha_beta_values('', candidate_song)
        (cx_alpha, cx_beta) = get_alpha_beta_values(song_3, candidate_song)
        (bcx_alpha, bcx_beta) = get_alpha_beta_values(song_2 + ',' + song_3, candidate_song)
        (abcx_alpha, abcx_beta) = get_alpha_beta_values(song_1 + ',' + song_2 +\
                                                        ',' + song_3, candidate_song)
        
        
        
        input_vector = np.log(np.array([x_alpha, cx_alpha, bcx_alpha, abcx_alpha, \
                  x_beta, cx_beta, bcx_beta, abcx_beta]) + 0.1) 
        
        
        alpha_value = 0.0
        beta_value = 1.0
        
        method_used = -1
        
        alpha_value = global_alpha.predict([ [input_vector] , [input_vector] ])[0][0]
        beta_value = global_beta.predict([ [input_vector] , [input_vector] ])[0][0]
        
        final_mean = (alpha_value)/(alpha_value+beta_value)

        to_return_map[candidate_song] = final_mean
        
    
    sorted_to_return = [k for k, v in 
                        sorted(to_return_map.items(), key=lambda item: item[1], reverse=True)]
    
    return sorted_to_return

In [ ]:
# Evaluating the performance on test data.

import numpy as np

correct_count = 0
total_count = 0

test_file = open('ml-1m/test.csv', 'r')
test_array = []

for line in test_file:
    songs_array = line.strip().split(',')
    test_array.append(songs_array)

bayesian_correct = 0
max_correct = 0

mrr_sum_bayesian = 0.0
mrr_sum_max = 0.0

recall_sum_bayesian = 0
recall_sum_max = 0

for test_playlist in test_array:

    if(len(test_playlist) < 23):
        continue
    
    if(total_count%100 == 0):
        print(total_count)
    
    #print('Currently processing: ' + str(total_count))
    
    song_1 = test_playlist[0]
    song_2 = test_playlist[1]
    song_3 = test_playlist[2]   
    
    correct_song = test_playlist[3]
    
    ground_truth_list = test_playlist[3:23]
    
    max_rec_list = most_freq_4gram(song_1, song_2, song_3)
    
    max_song = max_rec_list[0][0]
    
    # Computing the MRR@20 for Max Approach
    # First find rank
    current_rank = 0.0
    if(correct_song in max_rec_list):
        current_rank = 1.0/(max_rec_list.index(correct_song)+1.0)
    mrr_sum_max+=current_rank

    # Compute recall for max
    recall_sum_max += len(set(ground_truth_list).intersection(set(max_rec_list)))
    
    bayesian_rec_list = predict_correct_song(song_1, song_2, song_3, max_song)
    bayesian_rec_list_20 = bayesian_rec_list[:20]
    
    if(bayesian_rec_list_20[0] == 'i'):
        bayesian_rec_list_20 = bayesian_rec_list_20[1:]
    
    # Computing the MRR@20 for Bayesian
    # First find rank
    current_rank = 0.0
    if(correct_song in bayesian_rec_list_20):
        current_rank = 1.0/(bayesian_rec_list_20.index(correct_song)+1.0)
    mrr_sum_bayesian+=current_rank
    
    # Recall for Bayesian
    recall_sum_bayesian += len(set(ground_truth_list).intersection(set(bayesian_rec_list_20)))
    
    bayesian_predicted = bayesian_rec_list_20[0]
    
    # Computing the accuracy
    if(bayesian_predicted == correct_song):
        bayesian_correct+=1
    
    if(max_rec_list[0] == correct_song):
        max_correct+=1
        
    #print('======')    
    total_count+=1
    
print('All Done!')

In [ ]:
# Printing results for next-item accuracy
print('Bayesian: ' + str(bayesian_correct))
print('Max: ' + str(max_correct))
print('Total: ' + str(total_count))

In [ ]:
print('MRR Max Approach: ' + str(mrr_sum_max/total_count))
print('MRR Bayesian: ' + str(mrr_sum_bayesian/total_count))

print('=====================================')

print('Recall Max Approach: ' + str(recall_sum_max/(total_count*20)))
print('Recall Bayesian: ' + str(recall_sum_bayesian/(total_count*20)))